# Pandas チートシート

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## pd.merge(df_left, df_right, how='inner', on='key')
２つのDFを、共通部分で結合

## df.rename(columns={'A': 'a'}, index={'ONE': 'one'}
カラム名を新しい変更

## train[['x_0','y_0','z_0']]
 複数の列を表示するときはカラム名をリストにいれる

## ## train[['x_0','y_0','z_0']].values
DF の値のみほしいとき使える


## df.apply(lambda x: (x - x.mean()) / x.std())
各列に対して処理を行う

## df.groupby(['A', 'B']).sum()
A,B列でグルーピングして、各列で合計をとる

## df.loc[[1:3], ['Age', 'Height']]
1:3行,Age,Height列にアクセス locは名前による参照

## df.loc[[1:3], [0,2]]
1:3行　0,2行にアクセス　ilocは番号による参照



## df.loc[df["columns"]>=5]
DFはbool型のシーケンスを指定してTrueのみを取り出すフィルタリングができる  
df.loc[df['カラム']を含む条件式]  
二つの条件　df.loc[df["banana"]>=5][df['apple']<=3] も可能

## df.iloc[0:100,[0,2]]
 ilocは行番号と列番号で位置を指定できる
 0~100行、1,3列を取得

## df.dropna()
欠損値を持つ行を削除。df.dropna(axis=1)で列を削除  
how='all'ですべての列がNaNである行だけ削除。　thresh=4で非NaN値が4つ未満の行を削除　subset=['C']特定の列(C列)にNaNが含まれている行を削除

In [6]:
from io import StringIO
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df=pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [7]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [10]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


## df['C'].fillna(df['C'].mean())
## df.fillna("FILLNUM")
fillna()中の値で欠損値NaNを穴埋めする。平均値を入れいる場合は上のやり方、中央値は.median(),最貧値は.mode()を使う  
列を指定せずにdf.fillna(df.mean())とすれば、各列の平均値が欠損値に穴埋めされるので便利

In [16]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


## 参考 sklearnを用いた欠損値補完
### from sklearn.preprcessing import Imputer
### imr=Imputer(missing_value='NaN',strategy='mean',axis=1)
### imputed_data=imr.fit_transform(df.values)
一度、欠損値補完のインスタンスを生成しなければいけない。標準化StandardScalerを用いるときと同じ手順  
numpyの配列ndarrayを生成するので注意

In [21]:
from sklearn.preprocessing import Imputer
imr=Imputer(missing_values='NaN',strategy='mean',axis=1)
imputed_data=imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

## df.['col'].map(label_mapping)
要素を置換する。カテゴリ特徴量を数値データにする際に使える。  
置換されない値はNaNをとる。

In [66]:
df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


### 順序特徴量のマッピング
特徴量間に順序が明確に決まってる場合は、段階的に数値を当てはめても良い

In [68]:
size_mapping={'XL':3,'L':2,'M':1}
df['size']=df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


### 正解クラスラベルのエンコーディング
クラスラベルが数値出ないときに、対応させるように自分で変える。

In [69]:
# マッピング用の対応を作成
class_mapping={label:idx for idx,label in enumerate(df['classlabel'])}
print(class_mapping)

df['classlabel']=df['classlabel'].map(class_mapping)
df

{'class1': 2, 'class2': 1}


,color,size,price,classlabel
0,green,1,10.1,2
1,red,2,13.5,1
2,blue,3,15.3,2


## 参考　sklearnでのラベルエンコーディング
### from skleran.preprocessing import LabelEncoder
### class_le=LabelEncoder()
### y=class_le.fit_transform(df['classlabel'].values)
順序特徴量は反映されないので注意が必要

In [55]:
from sklearn.preprocessing import LabelEncoder
print(df['size'])
print(df['classlabel'])
class_le=LabelEncoder()
print(class_le.fit_transform(df['size']))
print(class_le.fit_transform(df['classlabel']))

0     M
1     L
2    XL
Name: size, dtype: object
0    class1
1    class2
2    class1
Name: classlabel, dtype: object
[1 0 2]
[0 1 0]


## pd.get_dummies(df[['price','color','size']])
one-hotエンコーディングを手軽に行ってくれる便利なライブラリ。特徴量が多くなりすぎる可能性があるので注意  

In [72]:
print(df)
df=pd.get_dummies(df[['color','size','price']])
print(df)

   color  size  price  classlabel
0  green     1   10.1           2
1    red     2   13.5           1
2   blue     3   15.3           2
   size  price  color_blue  color_green  color_red
0     1   10.1           0            1          0
1     2   13.5           0            0          1
2     3   15.3           1            0          0
